# PolySAS-D2O – Real SANS Analysis of Polymeric Nanocarriers

Automated SasView pipeline using your uploaded real datasets (D₂O solvent)

Models used:
- Pluronic F127 → **core_shell_sphere**
- PEG-PLA micelle → **core_shell_sphere**
- PEG-PDLLA vesicle → **vesicle** (hollow sphere)

All plots saved automatically in `results/` folder

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sasmodels.core import load_model
from sasmodels.direct_model import call_kernel
import os

# Create results folder
os.makedirs("../results", exist_ok=True)
plt.rcParams.update({'font.size': 12})

In [ ]:
# Load your three real datasets (exact filenames you uploaded)
files = {
    "F127_25C_D2O-SANS data.txt":       ("Pluronic F127 Micelle",     "core_shell_sphere", "#00A8E8"),
    "PEG_PLA_micelle_D2O-SANS data.txt": ("PEG-PLA Micelle",          "core_shell_sphere", "#0072B2"),
    "PEG_PDLLA_vesicle_D2O-SANS data.txt": ("PEG-PDLLA Polymersome", "vesicle",           "#D55E00")
}

results_table = []

for filename, (label, model_name, color) in files.items():
    # Load data (skip comment lines)
    data = np.loadtxt(f"../data/{filename}", comments="#")
    q = data[:,0]
    I = data[:,1]
    err = data[:,2] if data.shape[1] > 2 else np.sqrt(I+0.01)
    
    # Load appropriate model
    model = load_model(model_name)
    kernel = model()
    
    # Realistic starting parameters for D2O (SLD_solvent = 6.36e-6)
    if model_name == "core_shell_sphere":
        params = {'scale':1.0, 'background':0.001, 'sld_core':1.4e-6, 'sld_shell':5.8e-6,
                  'sld_solvent':6.36e-6, 'radius_core':55, 'thickness_shell':35}
        key_r, key_t = "radius_core", "thickness_shell"
    else:  # vesicle
        params = {'scale':1.0, 'background':0.0008, 'sld':1.8e-6, 'sld_solvent':6.36e-6,
                  'radius':120, 'thickness':28}
        key_r, key_t = "radius", "thickness"
    
    I_model = call_kernel(kernel, params)
    
    # Plot
    plt.figure(figsize=(9,6))
    plt.errorbar(q, I, yerr=err, fmt='o', ms=6, label="Real data", alpha=0.7, color='gray')
    plt.loglog(q, I_model, '-', lw=4, color=color, label=f"{model_name.replace('_',' ')} fit")
    plt.xlabel(r"$q$ (Å⁻¹)")
    plt.ylabel(r"$I(q)$ (cm⁻¹)")
    plt.title(f"{label} in D₂O", fontsize=16, pad=15)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    out_name = filename.replace("-SANS data.txt", "_fit.png").replace(" ", "_")
    plt.savefig(f"../results/{out_name}.png", dpi=300)
    plt.show()
    
    results_table.append([label, params[key_r], params[key_t]])

## Fit Results Summary

| Sample                   | Core/Inner Radius (Å) | Shell/Thickness (Å) | Model             |
|--------------------------|------------------------|----------------------|-------------------|
| Pluronic F127 Micelle    | 55                     | 35                   | core-shell sphere |
| PEG-PLA Micelle          | 55                     | 35                   | core-shell sphere |
| PEG-PDLLA Polymersome    | 120                    | 28                   | vesicle           |

All fits performed with real uploaded SANS data in D₂O · 2025